In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import timeit
from sklearn.model_selection import GridSearchCV,KFold

In [2]:
timeseries=pd.read_csv('Store-Sales-Time-Series.csv')
timeseries['date'] = pd.to_datetime(timeseries['date'])
timeseries

,date,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,2013-01-01,0.0,0.0,2.0,810.0,0.0,180.58900,0.0,186.0,143.0,...,0.0,110.801000,25.0,0.0,0.0,42.637000,37.847000,0.00000,0.0,0.000000
1,2013-01-02,255.0,0.0,207.0,72092.0,0.0,26246.31900,0.0,74629.0,23381.0,...,0.0,20871.464028,17204.0,0.0,0.0,13975.884938,5338.111976,0.00000,0.0,1526.750002
2,2013-01-03,161.0,0.0,125.0,52105.0,0.0,18456.48002,0.0,55893.0,18001.0,...,0.0,16597.398113,12568.0,0.0,0.0,10674.393983,3591.388005,0.00000,0.0,1094.310994
3,2013-01-04,169.0,0.0,133.0,54167.0,0.0,16721.96901,0.0,52064.0,18148.0,...,0.0,21625.963055,11303.0,0.0,0.0,10772.515038,4472.965990,0.00000,0.0,1293.120995
4,2013-01-05,342.0,0.0,191.0,77818.0,0.0,22367.76108,0.0,70128.0,23082.0,...,0.0,20879.091050,16819.0,0.0,0.0,13475.009055,5830.073020,0.00000,0.0,1245.637004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2017-08-11,441.0,7.0,351.0,189111.0,0.0,28204.48600,870.0,55682.0,44909.0,...,500.0,26944.092010,15978.0,587.0,654.0,25318.297990,5199.494021,118738.14300,3523.0,1272.615997
1680,2017-08-12,403.0,3.0,369.0,182318.0,0.0,27403.54800,849.0,57935.0,42854.0,...,483.0,17775.650112,15903.0,508.0,712.0,19134.510058,4573.465992,111788.35090,3644.0,1028.030006
1681,2017-08-13,481.0,9.0,433.0,202354.0,1.0,33262.75700,544.0,61757.0,50112.0,...,469.0,18633.015039,18188.0,541.0,741.0,20509.265004,4941.509018,125923.80240,3718.0,1118.047000
1682,2017-08-14,292.0,9.0,337.0,174832.0,0.0,28002.45700,594.0,55032.0,43050.0,...,457.0,16975.686040,15077.0,417.0,500.0,18597.508060,4647.375002,115257.59598,2826.0,970.679999


In [3]:
test_data=pd.read_csv('.gitignore/test.csv')
test_data

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [4]:
family_list=timeseries.columns
family_list=np.array(family_list)
family_list=family_list[1:]
forecast_number=int(len(test_data)/len(family_list))
print('Number of Forecast=',forecast_number)

Number of Forecast= 864


In [6]:
# Demonstration of the Sliding Window Method---Restructuring data for XGBRegressor
df =pd.DataFrame()
df['price'] = timeseries['AUTOMOTIVE']
df['target'] = df['price'].shift(-1)
df.dropna(inplace=True)
test=df[-1:]
train=df[:-1]
X=train['price']
y=train['target']
df

,price,target
0,0.0,255.0
1,255.0,161.0
2,161.0,169.0
3,169.0,342.0
4,342.0,360.0
...,...,...
1678,313.0,441.0
1679,441.0,403.0
1680,403.0,481.0
1681,481.0,292.0


In [16]:
# Finding best parameters for the model
xgb1 = XGBRegressor()
parameters = {'nthread':[4],
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500,1000]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X,
         y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[01:04:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[01:04:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


-0.6102771643354852
{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [24]:
# Application of the XGBRegressor

import warnings
warnings.filterwarnings('ignore')

start = timeit.default_timer()

prediction_df=pd.DataFrame()

prediction=[]
errors=[]
for a in family_list:
    #Sliding Window applied for each category
    df =pd.DataFrame()
    df['price'] = timeseries[a]
    df['target'] = df['price'].shift(-1)
    df.dropna(inplace=True)
    print(a+'--Completed ')
    predicted_values=[]
    error_list=[]
    for i in range(forecast_number):
        test=df[-1:]
        train=df[:-1]

        X=train['price']
        y=train['target']

        model=XGBRegressor(colsample_bytree=0.7, learning_rate=0.03, max_depth=5, min_child_weight=4, n_estimators=500, 
                           nthread=4, objective='reg:squarederror',subsample=0.7)
        model=model.fit(X,y)
        pred=model.predict(test['price'])
        
        # Walk-Forward Method -- Target value added on the price column, prediction value added on the target column
        predict=pd.DataFrame()
        predict['price']=test['target']
        predict['target']=pred.round(0)
        
        predicted_values.append(pred)
    
        df=df.append(predict,ignore_index=True) 

        error=mean_squared_error(test['target'],pred,squared=False)
        error_list.append(error)
        
    predicted_values.append(prediction)
    errors.append(error_list)
    
    prediction_df['Pred-'+a]=df['target'].loc[1683:]
    
stop = timeit.default_timer()

print('Time: ', stop - start)  

AUTOMOTIVE--Completed 
BABY CARE--Completed 
BEAUTY--Completed 
BEVERAGES--Completed 
BOOKS--Completed 
BREAD/BAKERY--Completed 
CELEBRATION--Completed 
CLEANING--Completed 
DAIRY--Completed 
DELI--Completed 
EGGS--Completed 
FROZEN FOODS--Completed 
GROCERY I--Completed 
GROCERY II--Completed 
HARDWARE--Completed 
HOME AND KITCHEN I--Completed 
HOME AND KITCHEN II--Completed 
HOME APPLIANCES--Completed 
HOME CARE--Completed 
LADIESWEAR--Completed 
LAWN AND GARDEN--Completed 
LINGERIE--Completed 
LIQUOR,WINE,BEER--Completed 
MAGAZINES--Completed 
MEATS--Completed 
PERSONAL CARE--Completed 
PET SUPPLIES--Completed 
PLAYERS AND ELECTRONICS--Completed 
POULTRY--Completed 
PREPARED FOODS--Completed 
PRODUCE--Completed 
SCHOOL AND OFFICE SUPPLIES--Completed 
SEAFOOD--Completed 
Time:  10037.904625399999


In [25]:
# Writting Predictions to the .csv file
prediction_df.to_csv('XGBOOST_2_Parameter_Est.csv')

In [26]:
# Sample Submission file is imported in order to convert our predictions to the submission format
samplesub_data=pd.read_csv('.gitignore/sample_submission.csv')
samplesub_data

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [27]:
prediction_df

,Pred-AUTOMOTIVE,Pred-BABY CARE,Pred-BEAUTY,Pred-BEVERAGES,Pred-BOOKS,Pred-BREAD/BAKERY,Pred-CELEBRATION,Pred-CLEANING,Pred-DAIRY,Pred-DELI,...,Pred-MAGAZINES,Pred-MEATS,Pred-PERSONAL CARE,Pred-PET SUPPLIES,Pred-PLAYERS AND ELECTRONICS,Pred-POULTRY,Pred-PREPARED FOODS,Pred-PRODUCE,Pred-SCHOOL AND OFFICE SUPPLIES,Pred-SEAFOOD
1683,321.0,10.0,279.0,142730.0,0.0,26700.0,683.0,60223.0,45755.0,14446.0,...,404.0,18251.0,13807.0,496.0,505.0,17994.0,5391.0,131767.0,3002.0,1160.0
1684,366.0,14.0,292.0,191124.0,0.0,27654.0,688.0,57546.0,45221.0,14392.0,...,362.0,19036.0,14089.0,369.0,566.0,17825.0,5343.0,140378.0,3141.0,1166.0
1685,315.0,11.0,232.0,172318.0,0.0,27448.0,672.0,62950.0,41734.0,14678.0,...,337.0,18125.0,14117.0,497.0,516.0,17455.0,5350.0,125426.0,1579.0,1086.0
1686,367.0,13.0,264.0,193031.0,0.0,24291.0,716.0,53636.0,46756.0,14835.0,...,371.0,17782.0,13332.0,354.0,602.0,17089.0,5334.0,136662.0,2437.0,1056.0
1687,339.0,13.0,201.0,163153.0,0.0,26910.0,785.0,66821.0,44051.0,14563.0,...,358.0,19267.0,13504.0,402.0,559.0,17404.0,5933.0,140183.0,1336.0,1078.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,364.0,12.0,228.0,194003.0,0.0,27490.0,912.0,57670.0,46053.0,15120.0,...,364.0,19416.0,14437.0,410.0,607.0,17462.0,5588.0,127465.0,1380.0,1061.0
2543,364.0,12.0,228.0,194109.0,0.0,25738.0,913.0,57694.0,46060.0,15123.0,...,364.0,19435.0,14432.0,410.0,607.0,17463.0,5591.0,127276.0,1380.0,1061.0
2544,364.0,12.0,228.0,194040.0,0.0,25554.0,914.0,57685.0,46067.0,15122.0,...,364.0,19426.0,14435.0,411.0,607.0,17470.0,5590.0,127287.0,1381.0,1062.0
2545,364.0,12.0,229.0,194048.0,0.0,26609.0,913.0,57684.0,46078.0,15132.0,...,364.0,19424.0,14434.0,412.0,607.0,17453.0,5589.0,127374.0,1381.0,1062.0


In [28]:
# Converting predictions to the submission format for Kaggle

prediction_df_index=list(prediction_df.index)
family_prediction_list=prediction_df.columns
submassion_list=[]
for i in prediction_df_index:
    for k in family_prediction_list:
        submassion_list.append(prediction_df[k][i])

In [29]:
sub_format=pd.DataFrame(samplesub_data['id'].values,columns=['id'],index=None)
sub_format['sales']=pd.DataFrame(submassion_list,columns=['sales'])
sub_format

,id,sales
0,3000888,321.0
1,3000889,10.0
2,3000890,279.0
3,3000891,142730.0
4,3000892,0.0
...,...,...
28507,3029395,17456.0
28508,3029396,5591.0
28509,3029397,127333.0
28510,3029398,1381.0


In [30]:
# Writting the last version of the submission format to the .csv file to submit it
sub_format.to_csv('XGBOOST_Submission_param_est.csv')